# BERT on Repo Description

1. Construct a sentence corpus for each software type using labeled/manually validated repo descriptions
2. Calculate embeddingfor each corpus
3. Compare all repo description with each software type corpus using cosine-similarity score
    - It took about 1.5 hrs to run the embedding on repo data

Author: Cierra and Crystal

In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
#pgadmin
import os
import psycopg2 as pg


#bert
from sentence_transformers import SentenceTransformer, util
import torch

import pandas as pd
import numpy as np

import re

import nltk
nltk.download("punkt")

from nltk import tokenize

import scipy

import datetime

[nltk_data] Downloading package punkt to /home/zz3hs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Embedding Model

In [4]:
#embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2') #quicker model
embedder = SentenceTransformer('paraphrase-mpnet-base-v2') #most accurate, long run time

# Data

## I. Unlabelled Repo Data

In [22]:
repo_data = pd.read_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/clean_github_repos_157k.csv")
len(repo_data)

157538

In [27]:
# get a list of repo descriptions
repo_description = repo_data["description"].tolist()

print(repo_description[0:10])
len(repo_description)

['  Vue js is a progressive  incrementally adoptable JavaScript framework for building UI on the web ', 'A declarative  efficient  and flexible JavaScript library for building user interfaces ', 'An Open Source Machine Learning Framework for Everyone', 'The most popular HTML  CSS  and JavaScript framework for developing responsive  mobile first projects on the web ', '   A delightful community driven  with 1700  contributors  framework for managing your zsh configuration  Includes nearly 300 optional plugins  rails  git  OSX  hub  capistrano  brew  ant  php  python  etc   over 140 themes to spice up your morning  and an auto update tool so that makes it easy to keep up with the latest updates from the community ', 'Flutter makes it easy and fast to build beautiful apps for mobile and beyond ', '  Algorithms and data structures implemented in JavaScript with explanations and links to further readings', 'JavaScript Style Guide', 'All Algorithms implemented in Python', ' Java   Java   Jav

157538

In [28]:
repo_description = [x for x in repo_description if type(x) != float]

In [30]:
len(repo_description)

155811

In [31]:
#15 cores, max memory
t1 = datetime.datetime.now()
print(t1)
repo_embedding = embedder.encode(repo_description, show_progress_bar=True, convert_to_tensor=True) 

t2 = datetime.datetime.now()
print(t2)
print(t2-t1)

2021-07-26 16:25:17.100569


Batches:   0%|          | 0/4870 [00:00<?, ?it/s]

2021-07-26 16:47:10.349418
0:21:53.248849


In [32]:
repo_embedding

tensor([[ 0.1056,  0.0480, -0.1564,  ...,  0.0483, -0.0313, -0.2305],
        [ 0.0476,  0.0466, -0.1291,  ...,  0.0327, -0.1107, -0.1609],
        [-0.1597,  0.2335, -0.0988,  ..., -0.0569, -0.1099, -0.1160],
        ...,
        [ 0.0803,  0.2915,  0.0199,  ..., -0.0829,  0.0010, -0.0423],
        [-0.0041,  0.2338, -0.0552,  ...,  0.0191,  0.1002, -0.1399],
        [-0.0163, -0.0915, -0.0162,  ...,  0.1489,  0.1094, -0.1212]])

In [33]:
np_repo_embedding = np.array(repo_embedding)

In [34]:
np_repo_embedding

array([[ 0.10562557,  0.04799023, -0.15635283, ...,  0.04826827,
        -0.0312915 , -0.23045011],
       [ 0.04755221,  0.04655255, -0.12910275, ...,  0.0326556 ,
        -0.1106855 , -0.16086306],
       [-0.1596753 ,  0.23353529, -0.09883387, ..., -0.05694686,
        -0.1099142 , -0.11601831],
       ...,
       [ 0.08030792,  0.29145524,  0.01986884, ..., -0.08285883,
         0.00104265, -0.04226664],
       [-0.00406473,  0.23381996, -0.05523785, ...,  0.01908189,
         0.10017424, -0.13986221],
       [-0.01630231, -0.0914646 , -0.01622014, ...,  0.14893341,
         0.1093612 , -0.1212249 ]], dtype=float32)

In [36]:

np.save('/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_description_embeddingsXXX.npy', np_repo_embedding)

## II. Labelled Repo Data -- Software Type Corpus

In [8]:
data = pd.read_excel('/home/zz3hs/git/dspg21oss/data/dspg21oss/labelled_repo/oss_software_labelled_python_sz.xlsx') #import csv
data

<ipython-input-8-fad580f16525>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  data = pd.read_excel('/home/zz3hs/git/dspg21oss/data/dspg21oss/labelled_repo/oss_software_labelled_python_sz.xlsx') #import csv


,slug,description,language,topics,commits,forks,stars,watchers,python_label,prog_python
0,openwrt/openwrt,this repository is a mirror of https://git.ope...,C,NaN,46330,5847,8944,547,NaN,1
1,grpc/grpc,"the c based grpc (c++, python, ruby, objective...",C++,NaN,42676,8069,30940,1359,1.0,1
2,libvirt/libvirt,read-only mirror. please submit merge requests...,C,NaN,35951,496,818,84,NaN,1
3,coreboot/coreboot,mirror of https://review.coreboot.org/coreboot...,C,NaN,32000,367,1309,116,NaN,1
4,ccxt/ccxt,a javascript / python / php cryptocurrency tra...,JavaScript,altcoin api arbitrage bitcoin bot cryptocurren...,27607,4971,20017,951,1.0,1
...,...,...,...,...,...,...,...,...,...,...
594,alfredfrancis/ai-chatbot-framework,a python chatbot framework with natural langua...,Python,chatbot python nltk ai sklearn natural-languag...,355,631,1491,130,1.0,1
595,jhao104/proxy_pool,pythonÁà¨Ëô´‰ª£ÁêÜipÊ±†(proxy pool),Python,crawler proxy proxypool spider ssdb flask craw...,354,3567,12785,438,NaN,1
596,getsentry/responses,a utility for mocking out the python requests ...,Python,NaN,354,269,3259,73,1.0,2
597,spotify/dh-virtualenv,python virtualenvs in debian packages,Python,dh-virtualenv python debian debian-packages om...,353,168,1471,41,NaN,1


In [14]:
# software type
type_name =  "python_label"
#filter 500 validated repos that are labelled 1 (numeric)
corpus_type_i = data[data[type_name] ==1][["slug",type_name]]


#perform a left merge to get cleaned repo description
corpus_type_i = corpus_type_i.merge(repo_data, on='slug', how='left')
corpus_type_i = corpus_type_i["description"].tolist()
corpus_type_i[0:10]

['The C based gRPC  C    Python  Ruby  Objective C  PHP  C  ',
 'A JavaScript   Python   PHP cryptocurrency trading API with support for more than 120 bitcoin altcoin exchanges',
 'Low Code Open Source Framework in Python and JS',
 'Flexible and powerful data analysis   manipulation library for Python  providing labeled data structures similar to R data frame objects  statistical functions  and much more',
 'Python based continuous integration testing framework  your pull requests are more than welcome ',
 'Interactive Data Visualization in the browser  from  Python',
 'Official repository for Spyder   The Scientific Python Development Environment',
 'NumPy aware dynamic Python compiler using LLVM',
 'A NumPy compatible array library accelerated by CUDA',
 'The Database Toolkit for Python']

# Embedding 

In [15]:
# embedding for the corpus
corpus_type_i_embeddings = embedder.encode(corpus_type_i, show_progress_bar=True) # embeddings


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [96]:

queries = repo_description

# pre-specified number of sentences
num_sentences = 10 #find 10 most similar sentences from the corpus

# init a result list for scores
result = []


t1 = datetime.datetime.now()
print("Start:", t1)

for query in queries: #compare each sentence in the abstract to the software type corpus
    #Compute embeddings
    query_embedding = embedder.encode(query, show_progress_bar=False, convert_to_tensor=True) 

    # We use cosine-similarity and torch.topk to find the highest k scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_type_i_embeddings)[0]

    top_results = torch.topk(cos_scores, k=num_sentences)   #get the top k scores
    result.append(top_results.values.tolist()) #unlist the top result list
   
    #print 10 most similar entences from the corpus and their corresponding scores
    #print("\n\n======================\n\n")
    #print("Query:", query)
    #print("Results:", top_results)
    #print("\nTop k=10 most similar sentences in corpus:")
    #for score, idx in zip(top_results[0], top_results[1]):
    #    print(corpus_type_i_clean[idx], "(Score: {:.4f})".format(score))

t2 =  datetime.datetime.now()
print("Finished", len(result), "descriptions at", t2)
print("It took", t2-t1, "to run.")

Start: 2021-07-17 15:15:06.160619
Finished 157538 descriptions at 2021-07-17 16:43:47.442722
It took 1:28:41.282103 to run.


In [97]:
#TODO: THIS WAY, similarity_score IS SAVED AS A STRING, NEED TO FIGURE OUT HOW TO SAVE AS A LIST
#save the similarity score as a variable of the original repo data
repo_data["similarity_score"] = result

In [98]:
#save csv
#repo_data.to_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_data_python_score.csv', index = False)   


# Similarity Score Analysis

In [18]:
from scipy import stats
from scipy.stats import skew
import statistics #calculate mean and others

In [16]:
#read in data
repo_data = pd.read_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/repo_data_python_score.csv')   


In [17]:
repo_data

,slug,description,readme,language,topics,commits,forks,stars,watchers,similarity_score
0,vuejs/vue,"🖖 Vue.js is a progressive, incrementally-adopt...","<p align=""center""><a href=""https://vuejs.org"" ...",JavaScript,"['vue', 'javascript', 'frontend', 'framework']",3070.0,29611.0,185611.0,6250.0,"[0.42707720398902893, 0.41736093163490295, 0.3..."
1,facebook/react,"A declarative, efficient, and flexible JavaScr...",# [React](https://reactjs.org/) · [![GitHub li...,JavaScript,"['javascript', 'react', 'frontend', 'declarati...",12695.0,34352.0,171327.0,6718.0,"[0.5678911805152893, 0.5504274368286133, 0.549..."
2,tensorflow/tensorflow,An Open Source Machine Learning Framework for ...,"<div align=""center"">\n<img src=""https://www.te...",C++,"['tensorflow', 'machine-learning', 'python', '...",75671.0,84937.0,156754.0,8092.0,"[0.6914295554161072, 0.6374238133430481, 0.602..."
3,twbs/bootstrap,"The most popular HTML, CSS, and JavaScript fra...","<p align=""center"">\n<a href=""https://getbootst...",JavaScript,"['css', 'bootstrap', 'javascript', 'html', 'sc...",19228.0,73981.0,151778.0,7079.0,"[0.4793936014175415, 0.46719563007354736, 0.46..."
4,ohmyzsh/ohmyzsh,🙃 A delightful community-driven (with 1700+ c...,"<p align=""center""><img alt=""Oh My Zsh"" src=""ht...",Shell,"['shell', 'zsh-configuration', 'theme', 'termi...",5447.0,22232.0,129314.0,2678.0,"[0.4871608316898346, 0.44735753536224365, 0.44..."
...,...,...,...,...,...,...,...,...,...,...
157533,VeryLittleGravitas/CDTADPQ,Very Little Gravitas implementation of Prototy...,"# CA Alerts, made with Very Little Gravitas fo...",CSS,"['prototype', 'messaging', 'emergency', 'govte...",414.0,1.0,0.0,2.0,"[0.34419283270835876, 0.3314574360847473, 0.32..."
157534,dajinchu/kde-connect-android,For Google Code-In,NaN,Java,[],414.0,0.0,0.0,1.0,"[0.525587260723114, 0.47061866521835327, 0.459..."
157535,LibrinnoTeam/LibraryHelpBot,Library Management System. ITP2 project,# Library Help Bot\r\n\r\n## Purpose of the ap...,Python,"['telegram-bot', 'mariadb', 'python', 'flask']",415.0,2.0,0.0,2.0,"[0.5185543894767761, 0.517078697681427, 0.4827..."
157536,Twissi/Animator,Animator for hacklace. See hacklace.org for fu...,Animator\n========\n\nAnimator for hacklace. S...,Java,[],415.0,0.0,0.0,0.0,"[0.41843315958976746, 0.4162086844444275, 0.40..."


In [29]:
#score is in a string, convert to a list, also make sure the numbers are float
score_ls = repo_data["similarity_score"]

score_ls_float = []
for sentence_score in score_ls:
    sentence_score = str(sentence_score)[1:-1]
    sentence_score = sentence_score.split(",")
    item_float= []
    for item in sentence_score:
        item_float.append(float(item))
    score_ls_float.append(item_float)

    
repo_data["similarity_score_float"] = score_ls_float

In [30]:
#check scores are in a list
repo_data["similarity_score_float"][0][0]

0.42707720398902893

In [31]:
# get score statistics
score_ls = repo_data["similarity_score_float"]

mean_score= []
range_score = []
max_score = []
median_score = []
skewness_score = []
for sentence_score in score_ls:
    mean_score.append(statistics.mean(sentence_score))
    range_score.append(max(sentence_score)- min(sentence_score))
    max_score.append(max(sentence_score))
    median_score.append(statistics.median(sentence_score))
    skewness_score.append(stats.skew(sentence_score))
repo_data["mean_score"]=mean_score
repo_data["range_score"]=range_score
repo_data["max_score"]=max_score
repo_data["median_score"]=median_score
repo_data["skewness_score"]=skewness_score

In [32]:
repo_data

,slug,description,readme,language,topics,commits,forks,stars,watchers,similarity_score,similarity_score_float,mean_score,range_score,max_score,median_score,skewness_score
0,vuejs/vue,"🖖 Vue.js is a progressive, incrementally-adopt...","<p align=""center""><a href=""https://vuejs.org"" ...",JavaScript,"['vue', 'javascript', 'frontend', 'framework']",3070.0,29611.0,185611.0,6250.0,"[0.42707720398902893, 0.41736093163490295, 0.3...","[0.42707720398902893, 0.41736093163490295, 0.3...",0.386763,0.065932,0.427077,0.381985,0.976499
1,facebook/react,"A declarative, efficient, and flexible JavaScr...",# [React](https://reactjs.org/) · [![GitHub li...,JavaScript,"['javascript', 'react', 'frontend', 'declarati...",12695.0,34352.0,171327.0,6718.0,"[0.5678911805152893, 0.5504274368286133, 0.549...","[0.5678911805152893, 0.5504274368286133, 0.549...",0.516635,0.074020,0.567891,0.503811,0.849749
2,tensorflow/tensorflow,An Open Source Machine Learning Framework for ...,"<div align=""center"">\n<img src=""https://www.te...",C++,"['tensorflow', 'machine-learning', 'python', '...",75671.0,84937.0,156754.0,8092.0,"[0.6914295554161072, 0.6374238133430481, 0.602...","[0.6914295554161072, 0.6374238133430481, 0.602...",0.603414,0.110041,0.691430,0.587026,1.810697
3,twbs/bootstrap,"The most popular HTML, CSS, and JavaScript fra...","<p align=""center"">\n<a href=""https://getbootst...",JavaScript,"['css', 'bootstrap', 'javascript', 'html', 'sc...",19228.0,73981.0,151778.0,7079.0,"[0.4793936014175415, 0.46719563007354736, 0.46...","[0.4793936014175415, 0.46719563007354736, 0.46...",0.424735,0.096729,0.479394,0.413202,0.336363
4,ohmyzsh/ohmyzsh,🙃 A delightful community-driven (with 1700+ c...,"<p align=""center""><img alt=""Oh My Zsh"" src=""ht...",Shell,"['shell', 'zsh-configuration', 'theme', 'termi...",5447.0,22232.0,129314.0,2678.0,"[0.4871608316898346, 0.44735753536224365, 0.44...","[0.4871608316898346, 0.44735753536224365, 0.44...",0.432683,0.075190,0.487161,0.425684,1.349496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157533,VeryLittleGravitas/CDTADPQ,Very Little Gravitas implementation of Prototy...,"# CA Alerts, made with Very Little Gravitas fo...",CSS,"['prototype', 'messaging', 'emergency', 'govte...",414.0,1.0,0.0,2.0,"[0.34419283270835876, 0.3314574360847473, 0.32...","[0.34419283270835876, 0.3314574360847473, 0.32...",0.317462,0.045936,0.344193,0.318741,0.451481
157534,dajinchu/kde-connect-android,For Google Code-In,NaN,Java,[],414.0,0.0,0.0,1.0,"[0.525587260723114, 0.47061866521835327, 0.459...","[0.525587260723114, 0.47061866521835327, 0.459...",0.366771,0.236830,0.525587,0.312547,0.700486
157535,LibrinnoTeam/LibraryHelpBot,Library Management System. ITP2 project,# Library Help Bot\r\n\r\n## Purpose of the ap...,Python,"['telegram-bot', 'mariadb', 'python', 'flask']",415.0,2.0,0.0,2.0,"[0.5185543894767761, 0.517078697681427, 0.4827...","[0.5185543894767761, 0.517078697681427, 0.4827...",0.471379,0.077404,0.518554,0.467471,0.701968
157536,Twissi/Animator,Animator for hacklace. See hacklace.org for fu...,Animator\n========\n\nAnimator for hacklace. S...,Java,[],415.0,0.0,0.0,0.0,"[0.41843315958976746, 0.4162086844444275, 0.40...","[0.41843315958976746, 0.4162086844444275, 0.40...",0.400396,0.032988,0.418433,0.397456,0.503637
